# Training Topic Model (LDA) on the Airbnb Comments

In [1]:

# import and setup modules we'll be using in this notebook
import logging
import itertools
import pandas as pd
import numpy as np
import gensim
import sys
reload(sys)
import cPickle
import re
sys.setdefaultencoding('utf8')
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO  # ipython sometimes messes up the logging setup; restore

In [2]:
from gensim.utils import smart_open, simple_preprocess
from gensim.corpora.wikicorpus import _extract_pages, filter_wiki
from gensim.parsing.preprocessing import STOPWORDS

def tokenize(text):
    return [token for token in simple_preprocess(text) if token.lower() not in STOPWORDS]

def iter_token(id_,comment):
    """Yield each article from the Wikipedia dump, as a `(title, tokens)` 2-tuple."""
  #  ignore_namespaces = 'Wikipedia Category File Portal Template MediaWiki User Help Book Draft'.split()
    for id_,comment in zip(id_,comment):
        tokens=tokenize(comment)
       # if len(tokens) < 50 :
        #    continue  # ignore short articles and various meta-articles
        yield id_, tokens
def person_remover(comment,person):
    return comment.replace(person,'')    

# Reading in data

In [3]:
col_to_read = ['listing_id','id','lan']
df_all_reviews= pd.read_csv('../DataFiles/merged_reviews_withCitiesLan.csv',usecols=col_to_read)

df_person = pd.read_csv('../DataFiles/sentences_withPersonName_NYC_.csv',header=None,sep='\t')
df_person.columns=['no','year','city','id','person','comment']

df_person=df_person.dropna()
df_person.year=df_person.year.astype(int)
df_person.id=df_person.id.astype(int)
df=pd.merge(df_person,df_all_reviews,on='id',how='left')
df=df[df.lan=='en']
df.listing_id.nunique()

27247

In [4]:
df['comment'] = df.apply(lambda row: person_remover(row['comment'], row['person']), axis=1)

df_text=df.groupby(['listing_id','year'])['comment'].apply(lambda x: '.'.join(x)).reset_index()
df_text=df_text.sort_values(['listing_id','year'], ascending=True)




In [5]:
#df_text=df_text[df_text.year>2009]
#df_=pd.DataFrame(df_text.groupby(['listing_id']).year.count())
#indices=df_[df_.year==7].index.tolist()
#df_text=df_text[df_text['listing_id'].isin(indices)]

In [6]:
df_text.to_csv('../DataFiles/sentences_withPersonName_NYC_listing.csv')

In [8]:
df_text=pd.read_csv('../DataFiles/sentences_withPersonName_NYC_listing.csv')

# Tokenizing documents

In [37]:
# Tokenize the documents.

from nltk.tokenize import RegexpTokenizer

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')

docs = list(df_text.dropna().comment)
for idx in range(len(docs)):
    if len(str(docs[idx]))<3:
        continue
    docs[idx]=re.sub(r'[^\x00-\x7F]+',' ', docs[idx])
    docs[idx] = docs[idx].lower()# Convert to lowercase.
    docs[idx] =tokenize(docs[idx])  # Split into words.

# Remove numbers, but not words that contain numbers.
#docs = [[token for token in doc if not token.isdigit()] for doc in docs]


# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]

In [38]:
from nltk.stem.wordnet import WordNetLemmatizer


            
# Lemmatize all words in documents.
lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

In [39]:
# Compute bigrams.

from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

In [40]:
# Remove rare and common tokens.

from gensim.corpora import Dictionary

# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)
dictionary.save('nyc_lda.dic')

with open(r"docs.pickle", "wb") as output_file:
    cPickle.dump(docs, output_file)

# Vectorizing documents, and save the corpus

In [41]:

# Vectorize data.

# Bag-of-words representation of the documents.
corpus = [dictionary.doc2bow(doc) for doc in docs]

with open(r"corpus.pickle", "wb") as output_file:
    cPickle.dump(corpus, output_file)

In [42]:
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

# Train and Save the LDA model

In [43]:
# Train LDA model.

from gensim.models import LdaModel

# Set training parameters.
num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

%time model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

model.save('lda.pkl')

# High frequent words in each topic

In [4]:
model = gensim.models.LdaModel.load('lda.pkl')
#top_topics = model.top_topics(corpus, num_words=5)
top_topics=model.show_topics()
top_topics

[(0,
  u'0.180*"family" + 0.121*"husband" + 0.037*"son" + 0.029*"mother" + 0.026*"mom" + 0.023*"brooklyn" + 0.022*"daughter" + 0.020*"park" + 0.017*"hidden" + 0.017*"house"'),
 (1,
  u'0.236*"new" + 0.195*"york" + 0.194*"new_york" + 0.077*"nyc" + 0.030*"village" + 0.029*"east" + 0.017*"west" + 0.013*"east_village" + 0.011*"upper" + 0.009*"west_village"'),
 (2,
  u'0.061*"restaurant" + 0.033*"recommendation" + 0.033*"area" + 0.027*"local" + 0.026*"neighborhood" + 0.025*"bar" + 0.019*"shop" + 0.018*"brooklyn" + 0.018*"eat" + 0.017*"nearby"'),
 (3,
  u'0.026*"arrived" + 0.019*"late" + 0.017*"day" + 0.015*"arrival" + 0.013*"met" + 0.012*"night" + 0.012*"let" + 0.011*"left" + 0.010*"provided" + 0.010*"got"'),
 (4,
  u'0.048*"easy" + 0.035*"question" + 0.033*"check" + 0.032*"helpful" + 0.032*"responsive" + 0.024*"accommodating" + 0.022*"communication" + 0.017*"quick" + 0.017*"key" + 0.015*"available"'),
 (5,
  u'0.150*"room" + 0.092*"nice" + 0.048*"roommate" + 0.044*"guy" + 0.043*"meet" + 0.

In [2]:
model = gensim.models.LdaModel.load('lda.pkl')
#top_topics = model.top_topics(corpus, num_words=5)
top_topics=model.show_topics()
# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
#avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
#print('Average topic coherence: %.4f.' % avg_topic_coherence)

topics=[]
top_topics=model.show_topics()
for t in top_topics:
    topic={}
    for chunk in t[1].split('+'):
        topic[chunk.split('*')[1].strip("'")] = float(chunk.split('*')[0])
    topics.append(topic)
    
from pprint import pprint
#pprint(top_topics)
df_list=[]

for topic in topics:

    df_= pd.DataFrame([(str(k),v) for k,v in topic.iteritems() ])
    df_.sort_values([1], ascending=False)
    
    df_list.append( df_[0])

df = pd.concat(df_list, axis = 1)
df.columns=[ [ 'Topic'+str(i) for i in range(len(df_list))]]
df

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Topic6,Topic7,Topic8,Topic9
0,"""mother""","""new""","""recommendation""","""let""","""available""","""cool""","""harlem""","""helpful""","""walk""","""readily_available"""
1,"""brooklyn""","""nyc""","""brooklyn""","""arrived""","""question""","""meet""","""food""","""home""","""manhattan""","""notice"""
2,"""mom""","""york""","""nearby""","""night""","""quick""","""roommate""","""morning""","""wonderful""","""station""","""welcome"""
3,"""park""","""west""","""neighborhood""","""late""","""helpful""","""chance""","""coffee""","""thank""","""block""","""job_making"""
4,"""family""","""west_village""","""bar""","""arrival""","""accommodating""","""nice""","""breakfast""","""time""","""minute""","""loft"""
5,"""daughter""","""new_york""","""area""","""day""","""check""","""living""","""house""","""welcoming""","""subway""","""short"""
6,"""son""","""east""","""local""","""got""","""responsive""","""bathroom""","""tea""","""nice""","""train""","""feel_welcome"""
7,"""house""","""village""","""restaurant""","""met""","""easy""","""friendly""","""mr""","""recommend""","""located""","""readily"""
8,"""husband""","""east_village""","""eat""","""left""","""key""","""room""","""kitchen""","""staying""","""location""","""feel"""
9,"""hidden""","""upper""","""shop""","""provided""","""communication""","""guy""","""delicious""","""friendly""","""close""","""short_notice"""


In [3]:
model.get_document_topics(corpus[0])

NameError: name 'corpus' is not defined